# 向量搜索基础教程
# Vector Search Basics Tutorial

这个笔记本将介绍向量搜索的基本概念和操作。

## 学习目标
- 理解向量的基本概念
- 学习向量操作和相似度计算
- 使用基本向量搜索功能

In [ ]:
# 导入必要的库
import sys
import os
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from src.basic_vector_search import BasicVectorSearch
from src.utils import visualize_vectors, plot_similarity_heatmap

## 1. 向量基础

向量是机器学习和信息检索的基础。让我们从简单的向量开始：

In [ ]:
# 创建一些示例向量
vector1 = np.array([1, 2, 3])
vector2 = np.array([4, 5, 6])
vector3 = np.array([1, 1, 1])

print("向量1:", vector1)
print("向量2:", vector2)
print("向量3:", vector3)

# 向量的维度
print(f"\n向量维度: {len(vector1)}")

## 2. 初始化向量搜索系统

In [ ]:
# 创建向量搜索实例
search_engine = BasicVectorSearch()

# 添加向量到索引
vectors = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [1, 1, 1],
    [2, 3, 4],
    [0, 1, 2]
])

search_engine.add_vectors(vectors)
print(f"已添加 {len(vectors)} 个向量到搜索引擎")
print(f"索引大小: {search_engine.get_index_size()}")

## 3. 相似度计算

让我们探索不同的相似度计算方法：

In [ ]:
# 查询向量
query_vector = np.array([2, 2, 2])

# 使用不同相似度度量进行搜索
print("查询向量:", query_vector)
print("\n=== 余弦相似度搜索 ===")
cosine_results = search_engine.search(query_vector, k=3, metric='cosine')
for i, (idx, similarity) in enumerate(cosine_results):
    print(f"{i+1}. 向量索引 {idx}, 相似度: {similarity:.3f}, 向量: {vectors[idx]}")

print("\n=== 欧几里得距离搜索 ===")
euclidean_results = search_engine.search(query_vector, k=3, metric='euclidean')
for i, (idx, distance) in enumerate(euclidean_results):
    print(f"{i+1}. 向量索引 {idx}, 距离: {distance:.3f}, 向量: {vectors[idx]}")

print("\n=== 曼哈顿距离搜索 ===")
manhattan_results = search_engine.search(query_vector, k=3, metric='manhattan')
for i, (idx, distance) in enumerate(manhattan_results):
    print(f"{i+1}. 向量索引 {idx}, 距离: {distance:.3f}, 向量: {vectors[idx]}")

## 4. 可视化

In [ ]:
# 可视化向量（仅使用前两个维度）
plt.figure(figsize=(10, 6))

# 绘制索引中的向量
plt.scatter(vectors[:, 0], vectors[:, 1], c='blue', s=100, label='索引向量', alpha=0.7)
for i, vec in enumerate(vectors):
    plt.annotate(f'V{i}', (vec[0], vec[1]), xytext=(5, 5), textcoords='offset points')

# 绘制查询向量
plt.scatter(query_vector[0], query_vector[1], c='red', s=150, label='查询向量', marker='*')
plt.annotate('Query', (query_vector[0], query_vector[1]), xytext=(5, 5), textcoords='offset points')

plt.xlabel('维度 1')
plt.ylabel('维度 2')
plt.title('向量分布可视化')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. 性能测试

In [ ]:
# 生成更大的数据集进行性能测试
large_vectors = np.random.rand(1000, 10)  # 1000个10维向量
large_search_engine = BasicVectorSearch()
large_search_engine.add_vectors(large_vectors)

# 性能基准测试
query = np.random.rand(10)
benchmark_results = large_search_engine.benchmark_search(query, k=10, num_queries=100)

print("性能基准测试结果:")
for metric, stats in benchmark_results.items():
    print(f"{metric}:")
    print(f"  平均时间: {stats['avg_time']:.4f}秒")
    print(f"  最小时间: {stats['min_time']:.4f}秒")
    print(f"  最大时间: {stats['max_time']:.4f}秒")
    print()

## 6. 总结

在这个笔记本中，我们学习了：

1. **向量基础**: 什么是向量以及如何表示
2. **向量搜索**: 如何构建和使用基本向量搜索系统
3. **相似度度量**: 不同相似度计算方法的差异
4. **可视化**: 如何可视化向量数据
5. **性能**: 搜索性能的测量和比较

下一个笔记本将深入探讨各种相似度度量的数学原理和应用场景。

# 向量搜索基础教程
# Vector Search Basics Tutorial

本教程将带您了解向量搜索的基础概念，包括:
- 向量表示
- 相似度度量
- 基础搜索算法

---

## 1. 导入必要的库

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple
import sys
import os

# 添加源代码路径
sys.path.append('../src')

# 设置绘图样式
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("✅ 库导入完成")

## 2. 什么是向量？

向量是数学中的基础概念，在机器学习和信息检索中，我们用向量来表示数据。

In [ ]:
# 创建简单的2D向量示例
vector_a = np.array([3, 4])
vector_b = np.array([1, 2])
vector_c = np.array([-2, 3])

print(f"向量 A: {vector_a}")
print(f"向量 B: {vector_b}")
print(f"向量 C: {vector_c}")

# 可视化向量
plt.figure(figsize=(8, 8))
plt.quiver(0, 0, vector_a[0], vector_a[1], angles='xy', scale_units='xy', scale=1, color='red', width=0.005, label='向量 A')
plt.quiver(0, 0, vector_b[0], vector_b[1], angles='xy', scale_units='xy', scale=1, color='blue', width=0.005, label='向量 B')
plt.quiver(0, 0, vector_c[0], vector_c[1], angles='xy', scale_units='xy', scale=1, color='green', width=0.005, label='向量 C')

plt.xlim(-3, 5)
plt.ylim(-1, 5)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linewidth=0.5)
plt.axvline(x=0, color='k', linewidth=0.5)
plt.legend()
plt.title('2D 向量可视化')
plt.xlabel('X 轴')
plt.ylabel('Y 轴')
plt.show()

# 计算向量的长度（L2范数）
print(f"\n向量长度:")
print(f"||A|| = {np.linalg.norm(vector_a):.3f}")
print(f"||B|| = {np.linalg.norm(vector_b):.3f}")
print(f"||C|| = {np.linalg.norm(vector_c):.3f}")

## 3. 相似度度量

相似度度量是向量搜索的核心。让我们学习几种常用的相似度计算方法。

In [ ]:
def cosine_similarity(v1, v2):
    """计算余弦相似度"""
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

def euclidean_distance(v1, v2):
    """计算欧几里得距离"""
    return np.sqrt(np.sum((v1 - v2) ** 2))

def manhattan_distance(v1, v2):
    """计算曼哈顿距离"""
    return np.sum(np.abs(v1 - v2))

def dot_product(v1, v2):
    """计算点积"""
    return np.dot(v1, v2)

# 计算向量间的相似度/距离
print("向量 A 和 B 之间的相似度/距离:")
print(f"余弦相似度: {cosine_similarity(vector_a, vector_b):.4f}")
print(f"欧几里得距离: {euclidean_distance(vector_a, vector_b):.4f}")
print(f"曼哈顿距离: {manhattan_distance(vector_a, vector_b):.4f}")
print(f"点积: {dot_product(vector_a, vector_b):.4f}")

print("\n向量 A 和 C 之间的相似度/距离:")
print(f"余弦相似度: {cosine_similarity(vector_a, vector_c):.4f}")
print(f"欧几里得距离: {euclidean_distance(vector_a, vector_c):.4f}")
print(f"曼哈顿距离: {manhattan_distance(vector_a, vector_c):.4f}")
print(f"点积: {dot_product(vector_a, vector_c):.4f}")

## 4. 相似度度量的几何解释

In [ ]:
# 创建更多向量来展示不同的相似度
angles = np.linspace(0, 2*np.pi, 8, endpoint=False)
vectors = np.array([[np.cos(angle), np.sin(angle)] for angle in angles])
reference_vector = np.array([1, 0])  # 参考向量

# 计算所有向量与参考向量的余弦相似度
similarities = [cosine_similarity(reference_vector, v) for v in vectors]

# 可视化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 左图: 向量可视化
ax1.quiver(0, 0, reference_vector[0], reference_vector[1], 
           angles='xy', scale_units='xy', scale=1, color='red', width=0.01, label='参考向量')

colors = plt.cm.viridis(np.linspace(0, 1, len(vectors)))
for i, (v, sim, color) in enumerate(zip(vectors, similarities, colors)):
    ax1.quiver(0, 0, v[0], v[1], angles='xy', scale_units='xy', scale=1, 
               color=color, width=0.005, label=f'向量 {i+1} (sim={sim:.3f})')

ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')
ax1.set_title('向量与参考向量的角度关系')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 右图: 相似度分布
ax2.bar(range(1, len(similarities)+1), similarities, color=colors)
ax2.set_xlabel('向量编号')
ax2.set_ylabel('余弦相似度')
ax2.set_title('余弦相似度分布')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("观察:")
print("- 余弦相似度衡量向量之间的角度")
print("- 相似度为1表示向量方向完全相同")
print("- 相似度为0表示向量垂直")
print("- 相似度为-1表示向量方向完全相反")

## 5. 高维向量示例

在实际应用中，我们通常处理高维向量（几十到几千维）。

In [ ]:
# 生成高维向量示例
np.random.seed(42)  # 保证结果可重现

# 创建一些高维向量
vector_dim = 100
n_vectors = 1000

# 方法1: 随机向量
random_vectors = np.random.rand(n_vectors, vector_dim)

# 方法2: 正态分布向量
normal_vectors = np.random.randn(n_vectors, vector_dim)

# 方法3: 稀疏向量（大部分元素为0）
sparse_vectors = np.zeros((n_vectors, vector_dim))
for i in range(n_vectors):
    # 只有10%的元素非零
    non_zero_indices = np.random.choice(vector_dim, size=vector_dim//10, replace=False)
    sparse_vectors[i, non_zero_indices] = np.random.randn(len(non_zero_indices))

print(f"创建了 {n_vectors} 个 {vector_dim} 维向量")
print(f"随机向量范围: [{np.min(random_vectors):.3f}, {np.max(random_vectors):.3f}]")
print(f"正态分布向量范围: [{np.min(normal_vectors):.3f}, {np.max(normal_vectors):.3f}]")
print(f"稀疏向量非零元素比例: {np.count_nonzero(sparse_vectors) / sparse_vectors.size:.1%}")

In [ ]:
# 比较不同向量类型的相似度分布
def compute_pairwise_similarities(vectors, sample_size=100):
    """计算成对相似度"""
    # 为了效率，只计算前sample_size个向量
    sample_vectors = vectors[:sample_size]
    similarities = []
    
    for i in range(sample_size):
        for j in range(i+1, sample_size):
            sim = cosine_similarity(sample_vectors[i], sample_vectors[j])
            similarities.append(sim)
    
    return similarities

# 计算相似度分布
random_sims = compute_pairwise_similarities(random_vectors)
normal_sims = compute_pairwise_similarities(normal_vectors)
sparse_sims = compute_pairwise_similarities(sparse_vectors)

# 可视化相似度分布
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].hist(random_sims, bins=30, alpha=0.7, color='blue')
axes[0].set_title('随机向量相似度分布')
axes[0].set_xlabel('余弦相似度')
axes[0].set_ylabel('频次')
axes[0].axvline(np.mean(random_sims), color='red', linestyle='--', label=f'均值: {np.mean(random_sims):.3f}')
axes[0].legend()

axes[1].hist(normal_sims, bins=30, alpha=0.7, color='green')
axes[1].set_title('正态分布向量相似度分布')
axes[1].set_xlabel('余弦相似度')
axes[1].set_ylabel('频次')
axes[1].axvline(np.mean(normal_sims), color='red', linestyle='--', label=f'均值: {np.mean(normal_sims):.3f}')
axes[1].legend()

axes[2].hist(sparse_sims, bins=30, alpha=0.7, color='orange')
axes[2].set_title('稀疏向量相似度分布')
axes[2].set_xlabel('余弦相似度')
axes[2].set_ylabel('频次')
axes[2].axvline(np.mean(sparse_sims), color='red', linestyle='--', label=f'均值: {np.mean(sparse_sims):.3f}')
axes[2].legend()

plt.tight_layout()
plt.show()

print("观察:")
print(f"- 随机向量平均相似度: {np.mean(random_sims):.4f}")
print(f"- 正态分布向量平均相似度: {np.mean(normal_sims):.4f}")
print(f"- 稀疏向量平均相似度: {np.mean(sparse_sims):.4f}")
print("\n随着维度增加，随机向量趋向于正交（相似度接近0）")

## 6. 基础向量搜索实现

现在让我们实现一个简单的向量搜索算法。

In [ ]:
class SimpleVectorSearch:
    def __init__(self):
        self.vectors = None
        self.labels = None
    
    def add_vectors(self, vectors, labels=None):
        """添加向量到搜索库"""
        self.vectors = np.array(vectors)
        if labels is None:
            self.labels = [f"向量_{i}" for i in range(len(vectors))]
        else:
            self.labels = labels
        print(f"添加了 {len(vectors)} 个向量")
    
    def search(self, query_vector, top_k=5):
        """搜索最相似的向量"""
        if self.vectors is None:
            raise ValueError("请先添加向量")
        
        # 计算查询向量与所有向量的相似度
        similarities = []
        for i, vector in enumerate(self.vectors):
            sim = cosine_similarity(query_vector, vector)
            similarities.append((i, sim, self.labels[i]))
        
        # 按相似度排序
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        return similarities[:top_k]

# 测试搜索功能
search_engine = SimpleVectorSearch()

# 使用前面生成的随机向量
sample_vectors = random_vectors[:100]  # 使用100个向量作为搜索库
sample_labels = [f"文档_{i:03d}" for i in range(100)]

search_engine.add_vectors(sample_vectors, sample_labels)

# 创建查询向量
query = np.random.rand(vector_dim)
print(f"\n查询向量维度: {len(query)}")

# 执行搜索
results = search_engine.search(query, top_k=5)

print("\n搜索结果 (Top 5):")
for rank, (idx, similarity, label) in enumerate(results, 1):
    print(f"{rank:2d}. {label}: {similarity:.4f}")

## 7. 向量归一化的重要性

In [ ]:
# 创建不同尺度的向量
vector_small = np.array([0.1, 0.2])
vector_large = np.array([10, 20])
vector_medium = np.array([1, 2])

print("原始向量:")
print(f"小向量: {vector_small}, 长度: {np.linalg.norm(vector_small):.3f}")
print(f"中向量: {vector_medium}, 长度: {np.linalg.norm(vector_medium):.3f}")
print(f"大向量: {vector_large}, 长度: {np.linalg.norm(vector_large):.3f}")

# 计算余弦相似度（注意它们方向相同）
print("\n余弦相似度（方向相同的向量）:")
print(f"小-中: {cosine_similarity(vector_small, vector_medium):.4f}")
print(f"小-大: {cosine_similarity(vector_small, vector_large):.4f}")
print(f"中-大: {cosine_similarity(vector_medium, vector_large):.4f}")

# 计算点积（受向量长度影响）
print("\n点积（受长度影响）:")
print(f"小-中: {np.dot(vector_small, vector_medium):.4f}")
print(f"小-大: {np.dot(vector_small, vector_large):.4f}")
print(f"中-大: {np.dot(vector_medium, vector_large):.4f}")

# 归一化向量
def normalize(v):
    return v / np.linalg.norm(v)

vector_small_norm = normalize(vector_small)
vector_medium_norm = normalize(vector_medium)
vector_large_norm = normalize(vector_large)

print("\n归一化后的向量:")
print(f"小向量: {vector_small_norm}, 长度: {np.linalg.norm(vector_small_norm):.3f}")
print(f"中向量: {vector_medium_norm}, 长度: {np.linalg.norm(vector_medium_norm):.3f}")
print(f"大向量: {vector_large_norm}, 长度: {np.linalg.norm(vector_large_norm):.3f}")

print("\n归一化后的点积（等于余弦相似度）:")
print(f"小-中: {np.dot(vector_small_norm, vector_medium_norm):.4f}")
print(f"小-大: {np.dot(vector_small_norm, vector_large_norm):.4f}")
print(f"中-大: {np.dot(vector_medium_norm, vector_large_norm):.4f}")

print("\n结论: 对于方向相同但长度不同的向量，归一化后点积等于余弦相似度")

## 8. 实践练习

让我们通过一个实际的例子来应用学到的知识。

In [ ]:
# 模拟文档向量搜索
# 假设我们有一些文档，每个文档用特征向量表示

# 创建模拟文档向量（每个维度代表一个词的重要性）
np.random.seed(123)
n_docs = 50
n_features = 20  # 简化的特征空间

# 文档类型: 科技、体育、艺术
doc_types = ['科技', '体育', '艺术']
doc_labels = []
doc_vectors = []

for i in range(n_docs):
    doc_type = doc_types[i % 3]
    
    if doc_type == '科技':
        # 科技文档在前7个特征上有较高值
        vector = np.concatenate([
            np.random.uniform(0.5, 1.0, 7),  # 科技相关特征
            np.random.uniform(0.0, 0.3, 13)  # 其他特征
        ])
    elif doc_type == '体育':
        # 体育文档在中间7个特征上有较高值
        vector = np.concatenate([
            np.random.uniform(0.0, 0.3, 7),   # 科技特征
            np.random.uniform(0.5, 1.0, 7),   # 体育相关特征
            np.random.uniform(0.0, 0.3, 6)    # 艺术特征
        ])
    else:  # 艺术
        # 艺术文档在后6个特征上有较高值
        vector = np.concatenate([
            np.random.uniform(0.0, 0.3, 14),  # 其他特征
            np.random.uniform(0.5, 1.0, 6)    # 艺术相关特征
        ])
    
    doc_vectors.append(vector)
    doc_labels.append(f"{doc_type}文档_{i:02d}")

doc_vectors = np.array(doc_vectors)

print(f"创建了 {n_docs} 个文档向量，每个 {n_features} 维")
print(f"文档类型分布: {len([l for l in doc_labels if '科技' in l])} 科技, "
      f"{len([l for l in doc_labels if '体育' in l])} 体育, "
      f"{len([l for l in doc_labels if '艺术' in l])} 艺术")

In [ ]:
# 可视化文档向量的特征分布
tech_docs = [i for i, label in enumerate(doc_labels) if '科技' in label]
sports_docs = [i for i, label in enumerate(doc_labels) if '体育' in label]
art_docs = [i for i, label in enumerate(doc_labels) if '艺术' in label]

plt.figure(figsize=(12, 8))

# 计算每类文档的平均特征值
tech_mean = np.mean(doc_vectors[tech_docs], axis=0)
sports_mean = np.mean(doc_vectors[sports_docs], axis=0)
art_mean = np.mean(doc_vectors[art_docs], axis=0)

plt.plot(tech_mean, 'o-', label='科技文档平均', color='blue')
plt.plot(sports_mean, 's-', label='体育文档平均', color='green')
plt.plot(art_mean, '^-', label='艺术文档平均', color='red')

plt.xlabel('特征维度')
plt.ylabel('特征值')
plt.title('不同类型文档的特征分布')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 创建搜索引擎并添加文档
doc_search = SimpleVectorSearch()
doc_search.add_vectors(doc_vectors, doc_labels)

In [ ]:
# 测试不同类型的查询
def test_query(query_vector, query_name):
    print(f"\n=== {query_name} ===")
    results = doc_search.search(query_vector, top_k=5)
    
    for rank, (idx, similarity, label) in enumerate(results, 1):
        print(f"{rank}. {label}: {similarity:.4f}")
    
    # 统计结果中各类型文档的数量
    tech_count = sum(1 for _, _, label in results if '科技' in label)
    sports_count = sum(1 for _, _, label in results if '体育' in label)
    art_count = sum(1 for _, _, label in results if '艺术' in label)
    
    print(f"结果分布: 科技={tech_count}, 体育={sports_count}, 艺术={art_count}")

# 查询1: 科技相关查询
tech_query = np.concatenate([
    np.ones(7) * 0.8,      # 科技特征高
    np.ones(13) * 0.1      # 其他特征低
])
test_query(tech_query, "科技查询")

# 查询2: 体育相关查询
sports_query = np.concatenate([
    np.ones(7) * 0.1,      # 科技特征低
    np.ones(7) * 0.8,      # 体育特征高
    np.ones(6) * 0.1       # 艺术特征低
])
test_query(sports_query, "体育查询")

# 查询3: 艺术相关查询
art_query = np.concatenate([
    np.ones(14) * 0.1,     # 其他特征低
    np.ones(6) * 0.8       # 艺术特征高
])
test_query(art_query, "艺术查询")

# 查询4: 混合查询
mixed_query = np.ones(20) * 0.5  # 所有特征中等
test_query(mixed_query, "混合查询")

## 9. 总结

在这个教程中，我们学习了:

1. **向量表示**: 如何用数值向量表示数据
2. **相似度度量**: 余弦相似度、欧几里得距离、曼哈顿距离等
3. **高维向量**: 实际应用中的向量特性
4. **向量搜索**: 基础的线性搜索算法
5. **归一化**: 向量归一化的重要性
6. **实践应用**: 文档搜索的简单示例

### 下一步

在接下来的教程中，我们将学习:
- 更高级的相似度度量方法
- 文本向量化技术
- 高效的近似搜索算法
- 实际应用案例

In [ ]:
# 练习: 尝试修改上面的代码，实验不同的参数设置
# 1. 改变文档特征分布
# 2. 尝试不同的相似度度量方法
# 3. 观察归一化对搜索结果的影响

print("🎉 教程完成！")
print("💡 建议: 尝试修改代码参数，观察结果变化")
print("📚 下一步: 学习 02_similarity_metrics.ipynb")